Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:


GaussianNB
LogisticRegression 
Linear + PolynomialFeatures.
RandomForestClassifier 
При обучении использовать make_pipeline
GridSearchCV для поиска параметров 
Посчитать матрики (f1 + точность + AUC). 



Откроем файл и создадим датафрейм для дальнейшей работы:

In [13]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, auc,roc_curve, roc_auc_score


with open('credit_train.csv', newline='', encoding='Windows-1251') as f:
      
    str = f.read().replace(",", "." ) 
    str = str.replace(";",",")    
    with open('train_ready.csv','w',  encoding='Windows-1251') as file2:    
        file2.write(str)
        
df = pd.read_csv("train_ready.csv",encoding='Windows-1251' ,on_bad_lines='skip',            
            usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
            )


Подготовка данных. Удалим слова Область, край и тд из Региона проживания:

In [2]:
bad_words = ['РЕСП','РЕСП.','РЕСПУБЛИКА', 'САХА', 'ОБЛ','АОБЛ', 'АВТОНОМНЫЙ ОКРУГ','ЮГРА', 'ЧУВАШИЯ', 'ФЕДЕРАЛЬНЫЙ ОКРУГ', 'АО', 'ОБЛАСТЬ', 'КРАЙ', 'Г', 'Ю', 'СЕВ.', 'СЕВЕРНАЯ', 'АЛАНИЯ', ' ']
change_dict = {'ОБЛАСТЬ':'','ОБЛ':'','КРАЙ':'','АО':'','АВТОНОМНЫЙ ОКРУГ':'','РЕСПУБЛИКА':'','РЕСП.':'','РЕСП':'','САХА/':'','САХА ':'','ЮГРА':'','-Ю':''}
print(len(df))
df = df.dropna()
print(len(df))
df = df.replace(to_replace =change_dict, regex = True)
df['living_region'] = df['living_region'].str.replace(" ", "")
df['living_region'] = df['living_region'].str.replace(r'[^а-яА-Яa-zA-Z\s]', '', regex=True)
df2 = df# for one hot encoder


170746
161331


Преобразуем категориальные переменные в  числовые методом LabelEncoder. Разделим данные методом TRAIN TEST SPLIT

In [ ]:
for col in ['gender', 'marital_status', 'job_position', 'education','living_region']:
    df[col] = LabelEncoder().fit_transform(df[col])

print(df)
# Разделим датафрейм на входные и выходные данные:
X = df.drop('open_account_flg', axis=1)
y = df['open_account_flg']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=30)

#Создадим список классификаторов:
classifiers = [
    RandomForestClassifier(n_estimators=30,max_features = 11) ,
    KNeighborsClassifier(n_neighbors = 100, weights ='distance',n_jobs = 4),
    GaussianNB(var_smoothing =2e-9),
    LogisticRegression(penalty = 'l2',C = 1.1, random_state=14,solver = 'newton-cholesky')        
    ]
    
# Переберем все классификаторы:
for clf in classifiers:
    clf.fit(X_train, y_train)
    print(" fit: ",type(clf))
    # Сделаем прогнозирование результата:
    y_pred = clf.predict(X_test)
    # Выведем вероятность нуля или единицы для вычисления AUC:
    y_score = clf.predict_proba(X_test)[:, 1]  
    # Рассчитаем fpr, tpr, для вычисления AUC:
    fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=None)
    # Рассчитаем  точность, ф1 меру и АУС метрику:
    print(f"{clf.__class__.__name__} accuracy: {accuracy_score(y_test, y_pred):.3f},  f1:  {f1_score(y_test, y_pred, average='micro')},  AUC:  {auc(fpr, tpr) }")


    


        client_id  gender   age  marital_status  job_position  credit_sum  \
7               8       0  26.0               3            12    47878.00   
9              10       0  32.0               3            12    26268.00   
10             11       1  26.0               2            12    16793.00   
12             13       1  37.0               2            12    42289.00   
13             14       1  43.0               2             5    57567.00   
...           ...     ...   ...             ...           ...         ...   
170741     170742       0  27.0               3            12    64867.00   
170742     170743       0  24.0               2            12    17640.00   
170743     170744       0  31.0               3            12    27556.47   
170744     170745       0  53.0               1             8     6189.00   
170745     170746       1  49.0               2            12    12787.00   

        credit_month  tariff_id  score_shk  education  living_region  \
7  

Выводы. 

RandomForestClassifier Работает медленно. По метрике AUC дает самый высокий результат. Увеличение количества деревьев более 30 не улучшает результат.
KNeighborsClassifier - на результат влияет только количество соседей. После 100 перестает давать эффект. Работает очень быстро.
GaussianNB   - результат слабый, работает очень быстро. Параметры не улучшают результат в сравнении с дефолтными.
LogisticRegression - серьезно улучшает результат AUC параметр solver = newton-cholesky

Создадим обработку ONEHOTENCODER. Все классификаторы здесь аналогичны предыдущему фрагменту. GaussianNB здесь не работает,
ругается на то, что матрица SPARCE, а не DENSE, что нормально для ONEHOTENCODER. Также здесь применим Pipeline.

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

#Разделим фичи на численные и категориальные. Создадим 2 пайплайна для каждого из них. 

numeric_features = ['client_id', 'age', 'credit_sum', 'credit_month','tariff_id', 'score_shk', 'monthly_income','credit_count','overdue_credit_count']
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ['gender', 'marital_status', 'job_position', 'education','living_region']
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        #("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
#Создадим препрроцессор для отдельной обработки численных и категориальных столбцов.
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)
# Создадим пайплайн для подстановки плассификаторов:
pipe = Pipeline(
    steps=[("preprocessor", preprocessor),"classifier", RandomForestClassifier(n_estimators=70)]
)
#Разделим датафрейм на входные и выходные параметры модели:
X = df2.drop('open_account_flg', axis=1)
y = df2['open_account_flg']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,shuffle=True, random_state=70)

# Создадим те же классификаторы, что и при использовании LabelEncoder
classifiers = [    
    RandomForestClassifier(n_estimators=30,max_features = 11) ,
    KNeighborsClassifier(n_neighbors = 100, weights ='distance',n_jobs = 4),
    #GaussianNB(var_smoothing =2e-9),
    LogisticRegression(penalty = 'l2',C = 1.1, random_state=14,solver = 'newton-cholesky')     
    ]

# Создадим цикл для перебора классификаторов.
for current_clf in classifiers:
    pipe.steps.insert(1,['classifier',current_clf])    
    clf = Pipeline(pipe.steps[0:2])    
    clf.fit(X_train, y_train)
    print(" fit: ",type(clf))
    y_pred = clf.predict(X_test)
    y_score = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=None)
    print(f"{current_clf.__class__.__name__} accuracy: {accuracy_score(y_test, y_pred):.3f},  f1:  {f1_score(y_test, y_pred, average='micro')},  AUC:  {auc(fpr, tpr) }")



 fit:  <class 'sklearn.pipeline.Pipeline'>
RandomForestClassifier accuracy: 0.835,  f1:  0.8353787033593653,  AUC:  0.7046758078510915
 fit:  <class 'sklearn.pipeline.Pipeline'>
KNeighborsClassifier accuracy: 0.838,  f1:  0.8383537870335936,  AUC:  0.6812985261740852
 fit:  <class 'sklearn.pipeline.Pipeline'>
LogisticRegression accuracy: 0.840,  f1:  0.8395934052311887,  AUC:  0.643713929954489


Выводы: Наилучший результат по AUC дал RandomForestClassifier.
По точности наилучший результат у LogisticRegression, однако в связи с неравномерно распределенным результатом на него нельзя ориентироваться.
Также стоит отметить, что ONEHOTENCODER по сравнению с LabelEncoder не дает улучшения параметров.


Так как метрики получены низкого качества, в качестве самопроверки применим код на выборке ирисов:

In [12]:
iris = pd.read_csv("iris.csv",encoding='Windows-1251' ,on_bad_lines='skip',
            #index_col='client_id' ,
            usecols=[0,1,2,3,4]
            )

iris['target'] = LabelEncoder().fit_transform(iris['target'])
X = iris.drop('target', axis=1)
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



classifiers = [
    RandomForestClassifier(n_estimators=30,max_features = 11) ,
    KNeighborsClassifier(n_neighbors = 100, weights ='distance',n_jobs = 4),
    GaussianNB(var_smoothing =2e-9),
    LogisticRegression(penalty = 'l2',C = 1.1, random_state=14,solver = 'newton-cholesky')        
]

for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_score = clf.predict_proba(X_test)[:, 1]    
   
    print(f"{clf.__class__.__name__} accuracy: {accuracy_score(y_test, y_pred):.3f},  f1:  {f1_score(yD_test, yD_pred, average='micro')}")
    
   

RandomForestClassifier accuracy: 1.000,  f1:  0.8351804841805461
KNeighborsClassifier accuracy: 1.000,  f1:  0.8351804841805461
GaussianNB accuracy: 1.000,  f1:  0.8351804841805461
LogisticRegression accuracy: 1.000,  f1:  0.8351804841805461


Выводы: Полученный код на выборке ирисов дает точность 100 процентов по причине простоты определени параметров. Для улучшения результата по выборке кредитного скоринга необходимо дальнейшая работа с классификаторами или их параметрами.